# Pre-Process Join MSSIS AIS and Carrier Vessels

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import fiona

from shapely.geometry import Point

In [ ]:
ais_file = '../data/interim/mssis-ais-records_v1.gpkg'
vessels_file = '../data/raw/vessels.csv'

out_path = '../data/interim/'

In [ ]:
gdf_ais = gpd.read_file(ais_file)
wgs84 = gdf_ais.crs

gdf_ais['ais_time'] = pd.to_datetime(gdf_ais['ais_time'], format='%Y-%m-%d %H:%M:%S')

# df_ais = df_ais[df_ais['vessel_mmsi']==211256150].sort_values(['ais_time'], ascending=True)

In [ ]:
gdf_ais.head()

In [ ]:
df_vessels = pd.read_csv(vessels_file, sep=',')

In [ ]:
df_vessels.head()

In [ ]:
df_vessels.fillna(0, inplace=True)

In [ ]:
df_vessels[['vessel_capacity_vehicle_units', 'vessel_stern_ramp_capacity_tons']] = df_vessels[['vessel_capacity_vehicle_units', 'vessel_stern_ramp_capacity_tons']].astype('int')

In [ ]:
df_vessels.head()

In [ ]:
cols = df_vessels.columns.tolist()[-6:]

In [ ]:
df_vessels['service_id'] = ''

for service in cols:
    df_vessels.loc[df_vessels[service] == 1, 'service_id'] = service
    df_vessels.drop(columns=[service], inplace=True)

In [ ]:
df_vessels

In [ ]:
join_col = np.intersect1d(df_vessels.columns, gdf_ais.columns).tolist()
join_col

In [ ]:
df = pd.merge(left=gdf_ais, right=df_vessels, how='outer', left_on=join_col, right_on=join_col)

In [ ]:
df.head()

In [ ]:
df = df.rename(columns = {'geometry': 'ais_geometry'}).set_geometry('ais_geometry')

In [ ]:
df.info()

In [ ]:
df.crs

In [ ]:
df.groupby(['mssis_wpi', 'mssis_eez', 'mssis_ao']).size().reset_index().rename(columns={0:'ais_count'}).sort_values(by=['ais_count'])

In [ ]:
df.groupby(['mssis_wpi','mssis_eez', 'mssis_ao']).size().reset_index().rename(columns={0:'ais_count'}).to_json(out_path + 'mssis-ais-region_v2.json', orient='records')

In [ ]:
# df[ df.mssis_eez == 'ATA']

In [ ]:
# d = df[ df.vessel_name == 'Philadelphia Express' ]

In [ ]:
# d[d.ais_time <= '2021-02-24 00:02:08'].sort_values(by=['ais_time'])

In [ ]:
# mssis_ais_records_location_count

In [ ]:
# mssis ={
#     'mssis_wpi': gdf.mssis_wpi.unique().tolist(),
#     'mssis_eez': gdf.mssis_eez.unique().tolist(),
#     'mssis_ao': gdf.mssis_ao.unique().tolist(),
#     'mssis_ais_records_location_count': mssis_ais_records_location_count
# }

In [ ]:
# with open(out_path + 'mssis-ais-region_v1.json', 'w') as json_file:
#     json.dump(mssis, json_file)

In [ ]:
# gdf.ais_time = gdf.ais_time.astype(str)
# gdf.info()

In [ ]:
df.to_file(out_path + 'mssis-ais-records_v2.geojson', driver='GeoJSON')

In [ ]:
df.to_file(out_path + 'mssis-ais-records_v2.gpkg', driver='GPKG')

# TODO: Delete Outlier Points